In [1]:
import trimesh
import numpy as np
from scipy.spatial import cKDTree

def sample_mesh(stl_path, num_points=10000):
    mesh = trimesh.load(stl_path)
    points, _ = trimesh.sample.sample_surface(mesh, num_points)
    return points

def sample_mesh_normalized(stl_path, num_points=10000):
    mesh = trimesh.load(stl_path)
    mesh.apply_translation(-mesh.centroid)
    scale = np.max(mesh.bounding_box.extents)
    mesh.apply_scale(1.0 / scale)
    points, _ = trimesh.sample.sample_surface(mesh, num_points)
    return points

def chamfer_distance(points1, points2):
    tree1 = cKDTree(points1)
    tree2 = cKDTree(points2)

    dist1, _ = tree1.query(points2)
    dist2, _ = tree2.query(points1)

    chamfer = np.mean(dist1**2) + np.mean(dist2**2)
    return chamfer


In [2]:
def f1_score(points_pred, points_gt, threshold=0.02):
    tree_pred = cKDTree(points_pred)
    tree_gt = cKDTree(points_gt)

    dist_pred_to_gt, _ = tree_gt.query(points_pred)
    precision = np.mean(dist_pred_to_gt < threshold)

    dist_gt_to_pred, _ = tree_pred.query(points_gt)
    recall = np.mean(dist_gt_to_pred < threshold)

    if precision + recall == 0:
        return 0.0

    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

In [3]:
import trimesh
import numpy as np

def normalize_mesh(mesh):
    """
    Normalize the mesh to fit inside a unit cube [0, 1]^3
    """
    mesh = mesh.copy()
    bounds = mesh.bounds
    scale = bounds[1] - bounds[0]
    max_extent = np.max(scale)
    mesh.apply_translation(-bounds[0])  # move to origin
    mesh.apply_scale(1.0 / max_extent)  # scale to fit into [0,1]
    return mesh

def voxelize_mesh(mesh, voxel_size=0.02):
    voxelized = mesh.voxelized(pitch=voxel_size)
    return voxelized.matrix 

def volumetric_iou(mesh1, mesh2, voxel_size=0.02):
    # Normalize both meshes to [0,1]^3
    mesh1 = normalize_mesh(mesh1)
    mesh2 = normalize_mesh(mesh2)

    # Voxelize
    vox1 = voxelize_mesh(mesh1, voxel_size)
    vox2 = voxelize_mesh(mesh2, voxel_size)

    # Pad to same shape
    shape = np.maximum(vox1.shape, vox2.shape)
    vox1_padded = np.zeros(shape, dtype=bool)
    vox2_padded = np.zeros(shape, dtype=bool)

    vox1_padded[:vox1.shape[0], :vox1.shape[1], :vox1.shape[2]] = vox1
    vox2_padded[:vox2.shape[0], :vox2.shape[1], :vox2.shape[2]] = vox2

    # Compute IOU
    intersection = np.logical_and(vox1_padded, vox2_padded).sum()
    union = np.logical_or(vox1_padded, vox2_padded).sum()

    if union == 0:
        return 1.0 if intersection == 0 else 0.0

    iou = intersection / union
    return iou


In [5]:
import os
candidate_list = []
candidate_dir = "./eval_result"

In [6]:
for candidate in os.listdir(candidate_dir):
    if candidate.startswith("gemma"):
        candidate_path = os.path.join(candidate_dir, candidate)
        with open(candidate_path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if line.endswith("Match: Yes"):
                    number = line.split(":")[0]
                    candidate_list.append(number)

In [7]:
inference_path = './stl'

In [8]:
from tqdm import tqdm

In [ ]:
cd_results = []
error_files = []
f1_results = []
iou_results = []
index = 0
for filename in tqdm(os.listdir(inference_path)):
    if filename.endswith('.stl') and not filename.startswith('._'):
        filename_index = filename.split(".")[0]
        if filename_index in candidate_list:
            ground_truth = f'../stlcq/{filename[:4]}/{filename}'
            prediction = f'./stl/{filename}'
            try:
                points_a = sample_mesh_normalized(ground_truth)
                points_b = sample_mesh_normalized(prediction)
                iou = volumetric_iou(trimesh.load(ground_truth), trimesh.load(prediction), voxel_size=0.02)
                iou_results.append(iou)
                f1 = f1_score(points_b, points_a)
                f1_results.append(f1)
                cd = chamfer_distance(points_a, points_b)
                cd_results.append(cd)
            except Exception as e:
                print(f"Error processing {filename}: {e}")
                error_files.append(filename)

In [11]:
import numpy as np

In [ ]:
print("F1 mean: ",np.mean(f1_results))
print("F1 median: ",np.median(f1_results))
print("CD mean: ",np.mean(cd_results)*1000)
print("CD median: ",np.median(cd_results)*1000)
print("IOU mean: ",np.mean(iou_results))
print("IOU median: ",np.median(iou_results))